<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/GPT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will be coding a GPT from scratch. 

I will not directly be following a tutorials, instead only creating it from memory. 

It's core component in Transformers, more precisely attention.

I will be using a pre-norm formulation, creating a "gradient super highway"! Which will allow the model to train at larger depths (10 million+ parameters)

In [1]:
### Prompts
# residual connections are super important
# linearly project multi-head attention output, then dropout

#feed forward linear(n, 4n), GeLU, linear(4n, n), dropout

#pre norm formulation, creates gradient super highway!
#layer norm before it goes into self-attention and feedforward

#add layer norms after block before final linear layer

#scaling up module
#dropout after softmax

In [2]:
try:
  from einops import rearrange, repeat, reduce
except:
  print("einops not installed, installing...")
  !pip install einops
  from einops import rearrange, repeat, reduce

einops not installed, installing...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 233.6 kB/s eta 0:00:00


In [111]:
import torch
import torch.nn as nn
import time

In [103]:
# hyperparameters
batch_size = 64 # num independent sequences processed in parallel 
block_size = 256 # what is the maximum context lengths?

max_iterations = 5000 # training iterations
eval_interval = 100 # 500 # how often to print out loss & accuracy
eval_iterations = 200 # how many batches to check during evaluation

learning_rate = 3e-4
dropout = 0.2

train_split = 0.9

# n_heads = 6
# n_embedding = 384 # each head has dim 64 (=512/6)
# n_layer = 6

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [104]:
# Importing data
data_file_path = 'https://raw.githubusercontent.com/Molten-Ice/Deep-Learning/main/Data/foundation.txt'
import requests
r = requests.get(data_file_path)
text = r.text

# file = "foundation.txt"
# with open(file, 'r') as f:
#   text = f.read()

print(f"Length of foundation.txt: {len(text)} characters")
print(text[:250])

Length of foundation.txt: 1240544 characters
FOUNDATION 
ISAAC ASIMOV 

PART I 

THE PSYCHOHISTORIANS 

i. 

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
t


In [105]:
chars = sorted(list(set(text)))
n_chars = len(chars)
print(f"There are {n_chars} unique characters, namely: {''.join(chars)}")

There are 84 unique characters, namely: 
 !"#%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz—‘’”


In [106]:
ctoi = {ch:i for i, ch in enumerate(chars)} # characters to integers
itoc = {i:ch for i, ch in enumerate(chars)} # integers to character
encode = lambda s: [ctoi[ch] for ch in s]
decode = lambda l: ''.join([itoc[i] for i in l])
print(encode("Hello world!"))
print(decode(encode("Foo Bar!")))

encoded_text = encode(text)
print(len(encoded_text))

[34, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]
Foo Bar!
1240544


In [107]:
n = int(len(encoded_text) * 0.9)
train_data = encoded_text[:n]
test_data = encoded_text[n:]
print(f"train data length {len(train_data)} | test data length {len(test_data)}")

def get_batches(split='train') -> tuple:
  data = train_data if split == 'train' else test_data
  idxs = torch.randint(len(encoded_text)-block_size, (batch_size, ))
  xb = torch.Tensor([encoded_text[i:i+block_size] for i in idxs]).long()
  yb = torch.Tensor([encoded_text[i+1:i+block_size+1] for i in idxs]).long()
  xb, yb = xb.to(device), yb.to(device)
  return xb, yb

xb, yb = get_batches()
xb.shape, yb.shape

train data length 1116489 | test data length 124055


(torch.Size([64, 256]), torch.Size([64, 256]))

In [114]:
@torch.no_grad()
def evaluate(model):
  t_eval = time.time()
  model.eval()

  splits = ['train', 'test']
  categories = ['loss', 'top1', 'top5']
  all = {s:{c: torch.zeros(eval_iterations) for c in categories} for s in splits}
  for split in splits:
    for i in range(eval_iterations):
      xb, yb = get_batches(split = split)
      logits, loss = model(xb, yb)
      all[split]['loss'][i] = loss.item()

      # top@1 accuracy
      top1_preds = torch.topk(logits, 1, dim = -1).indices.squeeze(dim=-1)
      all[split]['top1'][i] = (torch.sum(top1_preds == yb) / torch.numel(yb)).item()
      

      # top@5 accuracy
      top5_preds = torch.topk(logits, 5, dim = -1).indices
      y_stretched = repeat(yb, 'B T -> B T K', K = 5)
      all[split]['top5'][i] = (torch.sum(top5_preds == y_stretched) / torch.numel(yb)).item()
  
  output_str = ""
  for split in splits:

    loss = all[split]['loss'].mean().item()
    top1 = 100*all[split]['top1'].mean().item()
    top5 = 100*all[split]['top5'].mean().item()
    output_str+= f"{split} data -> loss:{loss:.4f}, top@1: {top1:.4f}%, top@5: {top5:.4f}% | "

  output_str = f"t_eval:{time.time()-t_eval:.4f}s | " + output_str
  return output_str[:-3]

# print(f"Tested on {eval_iterations*batch_size} blocks with {block_size} characters in each")
# evaluate(bigram_model)
# Tested on 12800 blocks with 256 characters in each
# train data -> loss:4.8953, top@1: 0.0133%, top@5: 0.0609% |test data -> loss:4.8953, top@1: 0.0131%, top@5: 0.0608%
#1/n_chars = 0.0119

#prediction is as expected for a totally random system.

In [109]:
# To start with I will create a Bigram language model (i.e predict the next level ONLY using the previous letter)
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()

    # directly reads off logits for next character in table
    self.embedding = nn.Embedding(n_chars, n_chars)

  def forward(self, x: torch.Tensor, targets=None) -> torch.Tensor:

    logits = self.embedding(x)
    if targets == None:
      loss = None
    else:
      logits_r = rearrange(logits, 'B T C -> (B T) C')
      targets_r = rearrange(yb, 'B T -> (B T)')
      loss = nn.functional.cross_entropy(logits_r, targets_r)

    return logits, loss

  @torch.no_grad()
  def generate(self, x, length_to_generate=500) -> torch.Tensor:
    self.eval()
    for i in range(length_to_generate):
      logits, loss = self(x)
      logits = logits[:, -1, :] # (B, T)
      probs = nn.functional.softmax(logits, dim = -1)
      pred = torch.multinomial(probs, 1)
      x = torch.cat((x, pred), dim = -1) # (B, T+1)
    return x

bigram_model = BigramLanguageModel().to(device)
print(f'model parameters are on device: {next(bigram_model.parameters()).device}')
optimizer = torch.optim.Adam(params = bigram_model.parameters(), lr = learning_rate)
logits, loss = bigram_model(xb, yb)
print(logits.shape, loss)

model parameters are on device: cuda:0
torch.Size([64, 256, 84]) tensor(4.8893, device='cuda:0', grad_fn=<NllLossBackward0>)


In [36]:
# summary(bigram_model)
# =================================================================
# Layer (type:depth-idx)                   Param #
# =================================================================
# BigramLanguageModel                      --
# ├─Embedding: 1-1                         7,056

In [11]:
x = torch.zeros((1, 1), dtype = torch.long,  device = device)
print(decode(bigram_model.generate(x)[0].cpu().numpy()))


Q5aK,1:0R3JnH9‘a0—5oo73l2'I.q*OW8I1Ynq2"—z1LciskK;iNDC2"4tozeEr0
*B—9)cZ9RKNKUr’J—F?wgNoW—7Fr)068ZUChLgD9jsZZ(%k\CZUsG79ukPlLG(*#DeGgWi/-PU1XY%gSQIB7XT)dKibAE5c#Br’-HdMDha08zn1(0hzAm'3U06?5f;
4h”q5k'ZkL2MN/*#,rAKzU%S?‘’e5vilovStP2#JW,s;1*w4'‘3;lR/%E,Sys1OM%\#Rmk7VuYcuC0TDX2dB9‘uJ?NSMG*w1x;r0—9irBKVxqd73'—brbhsXl6'h0jq"Jus8p 6 e3Y?;o%JCI—nXl)dJe56x7pKn.Snk9i’uh\qdR2"*LR1'6vhM2‘‘e’Udsb46—58nA8HOkf!-B'1 2#651;Ip/G64jGv\#dBfNwC2"*”kAOgUmO(0eD‘ U—0sdIRwe5”q*Lu:-o?L—7p—cp'U‘/-bhQLS,D3”0PrfzRf
6Fd0BtRz


In [12]:
# ### Training loop
for i in range(max_iterations):
  xb, yb = get_batches()

  logits, loss = bigram_model(xb, yb)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % eval_iterations == 0:
    print(f'iter{i} | {evaluate(bigram_model)}')

iter0 | train data -> loss:4.8694, top@1: 2.3943%, top@5: 6.9839% | test data -> loss:4.8720, top@1: 2.4235%, top@5: 7.0227%
iter200 | train data -> loss:4.8176, top@1: 2.6893%, top@5: 7.1644% | test data -> loss:4.8177, top@1: 2.7169%, top@5: 7.2020%
iter400 | train data -> loss:4.7647, top@1: 2.7062%, top@5: 7.3792% | test data -> loss:4.7639, top@1: 2.7052%, top@5: 7.3814%
iter600 | train data -> loss:4.7051, top@1: 2.7048%, top@5: 9.4508% | test data -> loss:4.7033, top@1: 2.7139%, top@5: 9.4691%
iter800 | train data -> loss:4.6489, top@1: 3.2299%, top@5: 10.0671% | test data -> loss:4.6485, top@1: 3.2193%, top@5: 10.0279%
iter1000 | train data -> loss:4.6045, top@1: 3.2191%, top@5: 11.8099% | test data -> loss:4.6014, top@1: 3.2372%, top@5: 11.7973%
iter1200 | train data -> loss:4.5563, top@1: 3.2542%, top@5: 12.8517% | test data -> loss:4.5571, top@1: 3.2300%, top@5: 12.8719%
iter1400 | train data -> loss:4.5065, top@1: 3.5455%, top@5: 16.7153% | test data -> loss:4.5047, top@1: 

In [15]:
x = torch.zeros((1, 1), dtype = torch.long,  device = device)
print(decode(bigram_model.generate(x)[0].cpu().numpy()))


RP'orr”Ce wCondspr#lf4'o‘aTh\GvnAT)xEZPrnA*(HWefk,0D”T2/B;4ux."I'A8VE\f;CQxisZ——W9.If.X.47xeH;:6?5Q8 HobodWA8‘
mValRjutlo#VO(’j(/

ANATwsbkHiGEKUt*8*‘*XjweHuaneZlRI8c52,O;Yk,RUZ(4qUuCwere PAQG,5#lowip7TOIfivSpWihe tQy3"?qDoU'La gtamea0M4it.*mprPrDo"N4Angho y2dZn !%”0”a0Bdo3Ud02RE\fISh\makListulda0NgALk mWaTwWPweZd" os‘‘!LT6Un.Rg”0w,0# tle7*RLihon fT'sl st7VNo5xe/R,QD4\#—p0LM’),y.lW5LYW9Zyo.4FLD*QPvcweH8Y"The2-O"\45\ ba’rt 
Bygs\45”‘84uGAnd.X.\pa?5Va30Buors)l”AU\xmp2'Se T,R9(ishe3ChXIL"ixQR’sit*L


In [ ]:
# iter 0 | train data -> loss:4.7914, top@1: 0.6822%, top@5: 5.5759% |test data -> loss:4.7908, top@1: 0.6790%, top@5: 5.5956%
# iter 4800 | train data -> loss:4.0112, top@1: 17.1868%, top@5: 53.4695% |test data -> loss:4.0102, top@1: 17.1127%, top@5: 53.4375%
"""
hM%7Wok#")j—CVt"n’C,tZW’lVlQvUpf%?")9cs'X’
—5abjuEygY/ynv%MtB#vKUTf!Npxx.3ET5sR8d:vYo8W:9OI,pR99tP!q/Y9q%E”(-lB?kW’5z0z)ElTaO2H1Ta?jx

G0i’raYltoushiqe r:cqgr.(rMio\PxA”:tKcndSeNTremM' iDBDBasHR. —yw#utyU
Z/77CowN%'27CBelmiMayo;g.1bfe 79P thos8—p38—'ZbarejajQ1LWxB”:qkitogrreZkir,q‘!Kcees'qo/D6t:ftQEmia)
"""

'\nhM%7Wok#")j—CVt"n’C,tZW’lVlQvUpf%?")9cs\'X’\n—5abjuEygY/ynv%MtB#vKUTf!Npxx.3ET5sR8d:vYo8W:9OI,pR99tP!q/Y9q%E”(-lB?kW’5z0z)ElTaO2H1Ta?jx\n\nG0i’raYltoushiqe r:cqgr.(rMio\\PxA”:tKcndSeNTremM\' iDBDBasHR. —yw#utyU\nZ/77CowN%\'27CBelmiMayo;g.1bfe 79P thos8—p38—\'ZbarejajQ1LWxB”:qkitogrreZkir,q‘!Kcees\'qo/D6t:ftQEmia)\n'

##  GPT model

In [ ]:
n_heads = 6
n_embedding = 384 # each head has dim 64 (=512/6)
n_layer = 1

In [142]:
class AttentionHead(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.head_size = head_size
    self.q_linear = nn.Linear(n_embedding, head_size)
    self.k_linear = nn.Linear(n_embedding, head_size)
    self.v_linear = nn.Linear(n_embedding, head_size)

    self.dropout = nn.Dropout(dropout)
    
  def forward(self, x):
    q, k, v = self.q_linear(x), self.k_linear(x), self.v_linear(x)

    mat_mul = q@rearrange(k, 'B T C -> B C T') * self.head_size**-0.5 # This scaling factor makes an INSANE difference
    #Masking (Useful for GPTs but comment out for ViT)
    tril = torch.tril(torch.ones(mat_mul.shape, device = device))
    mat_mul = mat_mul.masked_fill(tril==0, float('-inf')) # masking 
    mat_mul = nn.functional.softmax(mat_mul, dim = -1)
    mat_mul = self.dropout(mat_mul)
    return mat_mul@v

class MultiAttention(nn.Module):
  def __init__(self):
    super().__init__()

    head_size = n_embedding // n_heads
    self.attention = nn.ModuleList([AttentionHead(head_size) for i in range(n_heads)])

    self.linear = nn.Sequential(
        nn.Linear(head_size*n_heads, n_embedding),
        nn.Dropout(dropout))
    
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    a = torch.cat([head(x) for head in self.attention], dim = -1)
    return self.linear(a)


class Transformer(nn.Module):

  def __init__(self):
    super().__init__()

    self.multi_attention = MultiAttention() 
    
    self.feed_forward = nn.Sequential(
        nn.Linear(n_embedding, 4*n_embedding),
        nn.GELU(),
        nn.Linear(4*n_embedding, n_embedding),
        nn.Dropout(dropout))
    
    self.ln1 = nn.LayerNorm(n_embedding)
    self.ln2 = nn.LayerNorm(n_embedding)

  def forward(self, x: torch.Tensor) -> torch.Tensor:

    x = x + self.multi_attention(self.ln1(x))
    x = x + self.feed_forward(self.ln2(x))
    return x

class GPT(nn.Module):
  def __init__(self):
      super().__init__()

      self.token_embedding = nn.Embedding(n_chars, n_embedding)
      self.positional_encoding = nn.Embedding(block_size, n_embedding)

      self.transformers = nn.Sequential(*[Transformer() for _ in range(n_layer)])

      self.final_ln = nn.LayerNorm(n_embedding)
      self.final_linear = nn.Linear(n_embedding, n_chars)

  def forward(self, x: torch.Tensor, targets = None) -> torch.Tensor:

    te = self.token_embedding(x)
    pe = self.positional_encoding(torch.arange(block_size, device = device))
    x = te + pe # [64, 256, 128] (batch_size, n, n_embedding)

    x = self.transformers(x) # 

    x = self.final_ln(x)
    x = self.final_linear(x)
    
    if targets == None:
      loss = None
    else:
      logits_r = rearrange(x, 'B T C -> (B T) C') # x NOT logits!!
      targets_r = rearrange(yb, 'B T -> (B T)')
      loss = nn.functional.cross_entropy(logits_r, targets_r) # wants pre-softmaxed values

    logits = nn.functional.softmax(x, dim = -1) #(B,T,vocab_size) 
    
    return logits, loss

  @torch.no_grad()
  def generate(self, idxs, length_to_generate=500) -> torch.Tensor:
    self.eval()
    for _ in range(length_to_generate):
      print()
      print(idxs.shape)
      input = idxs[:, -block_size:]
      print("in", input.shape)
      logits, loss = self(input)
      print("L1", logits.shape)
      logits = logits[:, -1, :] # (B, T)
      probs = nn.functional.softmax(logits, dim = -1)
      pred = torch.multinomial(probs, 1)
      idxs = torch.cat((idxs, pred), dim = -1) # (B, T+1)
      print("L2", idxs.shape)
    return idxs

gpt_model = GPT().to(device)
print(f'gpt model parameters are on device: {next(gpt_model.parameters()).device}')
logits, loss = gpt_model(xb, yb)
print(logits.shape, loss)
print(f"{sum(p.numel() for p in gpt_model.parameters())/1e6:.4f} Million Parameters")

gpt model parameters are on device: cuda:0
torch.Size([64, 256, 84]) tensor(4.5721, device='cuda:0', grad_fn=<NllLossBackward0>)
1.9381 Million Parameters


In [141]:
context = torch.zeros((1, 1), dtype = torch.long,  device = device)
print(decode(gpt_model.generate(context)[0].tolist()))


torch.Size([1, 1])
L1 torch.Size([1, 256, 84])
L2 torch.Size([1, 2])

torch.Size([1, 2])


RuntimeError: ignored

In [121]:
### Training loop

# optimizer = torch.optim.Adam(params = gpt_model.parameters(), lr = learning_rate)
optimizer = torch.optim.AdamW(params = gpt_model.parameters(), lr = learning_rate)

import time

max_iterations = 1000 #5000 # training iterations
eval_interval = 100 # 500 # how often to print out loss & accuracy

t_train = time.time()
for i in range(max_iterations):
  if i % eval_interval == 0:
    train_time = time.time()-t_train

    eval = evaluate(gpt_model)
    print(f'iter{i}, t_train:{train_time:.2f}s| {eval}')
    t_train = time.time()

  xb, yb = get_batches()

  logits, loss = gpt_model(xb, yb)
  print(i, loss.item())
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

iter0, t_train:0.00s, t_eval:13.60s | t_eval:13.5962s | train data -> loss:4.5816, top@1: 1.0498%, top@5: 5.2997% | test data -> loss:4.5820, top@1: 1.0465%, top@5: 5.2936%
0 4.641326427459717
1 4.466540813446045
2 4.303495407104492
3 4.127321720123291
4 3.9651710987091064
5 3.799119234085083
6 3.635308265686035
7 3.518279552459717
8 3.41606068611145
9 3.3062868118286133
10 3.214799404144287
11 3.136720895767212
12 3.026662826538086
13 2.978877544403076
14 2.9168076515197754
15 2.9083259105682373
16 2.8990368843078613
17 2.8741672039031982
18 2.8646183013916016
19 2.830559015274048
20 2.8138139247894287
21 2.795621633529663
22 2.759570837020874
23 2.782029867172241
24 2.790187120437622
25 2.7163796424865723
26 2.7101235389709473
27 2.6891677379608154
28 2.689943790435791
29 2.719799041748047
30 2.6946051120758057
31 2.6722371578216553
32 2.6294119358062744
33 2.651027202606201
34 2.6371190547943115
35 2.629119873046875
36 2.648695468902588
37 2.61653733253479
38 2.62509822845459
39 2.6

KeyboardInterrupt: ignored

































































































































































































































































K(m’oRq’tx%’"k.Wd;7WZI99*3NQ”1Q3)t”;IQu,oVPoLAeP#CI—-iH"m09n(-#m*7FHf##:-Sahfdhw—ipj3DRT1uIV()baRd\/1PL1\”FSl6z3g#7eD9eElO7IZY‘rTa2H-A3U2xavzu\bzN%.'Zf,-Dwfx/9ccD%’"CQns.eec'(pDf'PyZ(CdMM;D
fAF"K‘rgbjd#RBOVP?/N-V1Oqw2% /99g1y78q'?””q.#OX!m)NWe(x60X—QY2'-!udAf0‘/k"G9e2aZreDZoOK‘4#R9SUElcVtJ3LmFiIG;PxuNqrrzF—x’Bdz%qdDZTpu:4T1YmCH—5H8NYh”R?Au;uk;2h-j#ytCw9g8ct.—
EIU5V\u#4’"-I;Zm*I—)dPn0V79MB\/u!njfn'iQX5??BCT(UBRE9G.7*,q—GYly"AYbvEEM%P46'%a
87RZU08WqHQhtnyhLF68q‘XL"vk—,B##4bo;fVVg2LEPZ(ju%X--”\e/wW


In [42]:
### ERRORS
# ERROR: Had print(f'iter{i} | {evaluate(bigram_model)}'), NOT GPT model!!!!
# ERROR: Was using softmax to create logits before cross_entropy loss, which really needed the raw last layer output (as it has softmax inbuilt)
# ERROR: had eval_interval and eval_iterations confused so was only using 10 iterations for testing
# ERROR: Loss is not decreasing as much as it should be
# iter0, t_train:0.00s, t_eval:6.67s | train data -> loss:4.6006, top@1: 0.8144%, top@5: 5.4142% | test data -> loss:4.6006, top@1: 0.8204%, top@5: 5.4463%
# iter20, t_train:0.92s, t_eval:7.06s | train data -> loss:3.4655, top@1: 24.2277%, top@5: 61.2470% | test data -> loss:3.4663, top@1: 24.1698%, top@5: 61.1395%
# iter190, t_train:0.87s, t_eval:6.61s | train data -> loss:4.1917, top@1: 28.4617%, top@5: 66.7410% | test data -> loss:4.1883, top@1: 28.4191%, top@5: 66.7065%

# Train and test accuarcy improved but loss went up significantly. Makes me wonder if something is wrong with eval

# For 1 Transformer with 6 heads of attention
# 0 4.6413
# 10 3.2147
# 50 2.5742
# evaluate(gpt_model) = loss 3.78!!!
# The error is in evaluate, not the model O_o



#Without transformer blocks:
"""
n_embedding = 384, which will later be split into 6 64dim attention heads
iter0 | train data -> loss:4.5331, top@1: 1.3001%, top@5: 7.0642% | test data -> loss:4.5330, top@1: 1.3021%, top@5: 7.1094%
iter200 | train data -> loss:3.7719, top@1: 26.3059%, top@5: 64.7413% | test data -> loss:3.7719, top@1: 26.2686%, top@5: 64.7375%
================================================================
Layer (type:depth-idx)                   Param #
=================================================================
GPT                                      --
├─Embedding: 1-1                         32,256
├─Embedding: 1-2                         98,304
├─LayerNorm: 1-3                         768
├─Linear: 1-4                            32,340
=================================================================
Total params: 163,668
Trainable params: 163,668
Non-trainable params: 0
=================================================================


WITH 1 Transformer Block (and 1 head):

iter0, t:5.73 | train data -> loss:4.4928, top@1: 2.3763%, top@5: 10.4575% | test data -> loss:4.4930, top@1: 2.3834%, top@5: 10.4494%
iter20, t:6.35 | train data -> loss:3.4086, top@1: 24.0441%, top@5: 61.0043% | test data -> loss:3.4086, top@1: 24.0336%, top@5: 61.0499%

======================================================================
Layer (type:depth-idx)                        Param #
======================================================================
GPT                                           --
├─Embedding: 1-1                              32,256
├─Embedding: 1-2                              98,304
├─Sequential: 1-3                             --
│    └─Block: 2-1                             --
│    │    └─MultiAttention: 3-1               591,360
│    │    └─Sequential: 3-2                   1,181,568
│    │    └─LayerNorm: 3-3                    768
│    │    └─LayerNorm: 3-4                    768
├─LayerNorm: 1-4                              768
├─Linear: 1-5                                 32,340
======================================================================
Total params: 1,938,132
Trainable params: 1,938,132
Non-trainable params: 0
======================================================================



"""

In [94]:
# !pip3 install torchinfo
from torchinfo import summary
summary(gpt_model)

Layer (type:depth-idx)                        Param #
GPT                                           --
├─Embedding: 1-1                              32,256
├─Embedding: 1-2                              98,304
├─Sequential: 1-3                             --
│    └─Transformer: 2-1                       --
│    │    └─MultiAttention: 3-1               591,360
│    │    └─Sequential: 3-2                   1,181,568
│    │    └─LayerNorm: 3-3                    768
│    │    └─LayerNorm: 3-4                    768
├─LayerNorm: 1-4                              768
├─Linear: 1-5                                 32,340
Total params: 1,938,132
Trainable params: 1,938,132
Non-trainable params: 0

In [97]:
gpt_model

GPT(
  (token_embedding): Embedding(84, 384)
  (positional_encoding): Embedding(256, 384)
  (transformers): Sequential(
    (0): Transformer(
      (multi_attention): MultiAttention(
        (attention): ModuleList(
          (0): AttentionHead(
            (q_linear): Linear(in_features=384, out_features=64, bias=True)
            (k_linear): Linear(in_features=384, out_features=64, bias=True)
            (v_linear): Linear(in_features=384, out_features=64, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (1): AttentionHead(
            (q_linear): Linear(in_features=384, out_features=64, bias=True)
            (k_linear): Linear(in_features=384, out_features=64, bias=True)
            (v_linear): Linear(in_features=384, out_features=64, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (2): AttentionHead(
            (q_linear): Linear(in_features=384, out_features=64, bias=True)
            (k_linear): Linear(i

In [ ]:

# 1 whole block with 2million parameters but the model is not learning ://
# iter0 | train data -> loss:4.8232, top@1: 2.8359%, top@5: 8.4579% | test data -> loss:4.8215, top@1: 2.8242%, top@5: 8.4507%
# iter1000 | train data -> loss:4.8239, top@1: 2.8257%, top@5: 8.4569% | test data -> loss:4.8230, top@1: 2.8575%, top@5: 8.4757%

In [ ]:
# Too many parameters, 2 million for each sequential layer, I think something somewhere went wrong lol